# ES|QL concepts

## Resources

* [Blog post announcement](https://www.elastic.co/blog/esql-elasticsearch-piped-query-language)
* [Documentation](https://www.elastic.co/docs/explore-analyze/query-filter/languages/esql)
* [Reference](https://www.elastic.co/docs/reference/query-languages/esql)
* Webinar: [ES|QL: Search. Aggregate. Transform. Visualize. All with one query](https://www.elastic.co/virtual-events/cd-esql-search-aggregate-transform-visualize-all-with-one-query)
* Technical blog posts in [Search Labs](https://www.elastic.co/search-labs/blog/category/esql)
* [Run this notebook in Google Colaboratory](https://colab.research.google.com/github/jsanz/foss4g_europe_lab/blob/main/02-esql.ipynb)

## Setup

In [1]:
# Install required dependencies using uv, if available, or directly with pip
!pip install -qU elasticsearch geopandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 929.5/929.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 kB 2.2 MB/s eta 0:00:00


Start with the necessary imports, a couple tweaks, and defining a "esql" query that will make easier to see the results of a query, by generating a Pandas or Geopandas dataframe.

In [2]:
import os
import io

import warnings

from elasticsearch import Elasticsearch
from elasticsearch import ElasticsearchWarning
from elasticsearch.exceptions import BadRequestError

import pandas as pd
import geopandas as gpd
from shapely import wkb

# Hide the warning when no LIMIT is passed on a ES|QL query
warnings.filterwarnings('ignore', category=ElasticsearchWarning)

# Allow wide columns
pd.set_option('display.max_colwidth', None)

# Convert Well-known Binary to Text
def wkb_to_wkt(wkb_bytes):
    if wkb_bytes is None:
        return None
    try:
        return wkb.loads(wkb_bytes).wkt
    except Exception as e:
        print(f"Error converting WKB: {wkb_bytes} - {e}")
        return None

# Generate a Pandas Dataframe or a Geopandas Dataframe from a ES|QL query
def esql(query, geometry_col:str = "geometry", use_arrow:bool = False):
    try:
        # Query ES and create a Pandas Dataframe
        if use_arrow:
            es_response = client.esql.query(query=query.strip(), format="arrow", columnar=True)
            df = es_response.to_pandas()
        else:
            es_response = client.esql.query(query=query.strip(), format="csv")
            df = pd.read_csv(io.StringIO(str(es_response)))

        # Promote to a Geopandas Dataframe if a "geometry" column
        if geometry_col in df.columns:
            if use_arrow:
                # Arrow geometries are transferred as WKB
                df[geometry_col] = df[geometry_col].apply(wkb_to_wkt)
            gs = gpd.GeoSeries.from_wkt(df[geometry_col])
            gdf = gpd.GeoDataFrame(df, geometry=gs, crs="EPSG:4326")
            if geometry_col != "geometry":
                gdf.drop(columns="geometry")
            return gdf
        else:
            return df
    except BadRequestError as e:
        print("Something went wrong!")
        print(e.message)
        print("\r\n".join([c['reason'] for c in e.info['error']['root_cause']]))

Connect to Elasticsearch and print some cluster details

In [7]:
# Login details
ES_URL=os.getenv("ES_URL","https://siglibre2025.es.us-central1.gcp.cloud.es.io")
KB_URL=os.getenv("KB_URL","https://siglibre2025.kb.us-central1.gcp.cloud.es.io")

# API key that allows reading indices
ES_APIKEY=os.getenv("ES_APIKEY", "NUF3RDlwZ0IxNDhsNzFFOXFDRWs6VS1xT1NKbTlGckdiMXdxVlBVd3RiQQ==")

# Load the client
client = Elasticsearch(hosts=[ES_URL], api_key=ES_APIKEY)

# Check the client
if client.ping():
  print("Connected to Elasticsearch")
  c_info = client.info()
  is_serverless = c_info['version']['build_flavor'] == 'serverless'

  # Print some cluster details
  print(f"Elasticsearch URL: {ES_URL}")
  print(f"Cluster name: {c_info['name']}")
  print(f"Version: {c_info['version']['number'] if not is_serverless else 'serverless'}")
  print("Number of documents indexed: ", client.count(index="*")['count'])
else:
  print("Connection failed")


Connected to Elasticsearch
Elasticsearch URL: https://siglibre2025.es.us-central1.gcp.cloud.es.io
Cluster name: instance-0000000001
Version: 9.1.3
Number of documents indexed:  516348


## Syntax and API

### Basic syntax

<https://www.elastic.co/docs/reference/query-languages/esql/esql-syntax>

A ES|QL query is made of a `source` command that sets the data to retrieve and a list of processing commands starting with the pipe `|` character.

```text
source-command
| processing-command1
| processing-command2
```

A query can contain one line and multi line comments.

```
source-command           // Single line comment
| processing-command1    // another comment
/*
a multi
line comment in between
processing commands
*/
| processing-command2
```

About literals:

* Literals are duble quoted
* If a double quote is required in a literal, triple quotes can be used

```text
ROW name = """Indiana "Indy" Jones"""
```


### Query API

* [Documentation](https://www.elastic.co/docs/explore-analyze/query-filter/languages/esql-rest)
* [Reference](https://www.elastic.co/docs/api/doc/elasticsearch/group/endpoint-esql)


Elasticsearch exposes the `_query` endpoint to execute ES|QL queries, with a `format` parameter to select between different output types as `csv`, `tsv`, `arrow`, `json`, etc.

As a `curl` command a request would be as:

```
curl -X POST \
  -H "Authorization: ApiKey $ES_APIKEY" \
  -H "Content-Type: application/json" \
  -d '{"query":"FROM places-* | STATS count = count(name) | LIMIT 1"}'\
  "$ES_URL/_query?pretty&format=txt"

     count     
---------------
230573
```

In the [Kibana Console](https://www.elastic.co/docs/explore-analyze/query-filter/tools/console):

```
POST /_query?format=txt
{
  "query": "FROM places-* | STATS count = count(name) | LIMIT 1"
}
```

And sending multiline queries is possible with triple quotes:

```text
POST /_query?format=txt
{
  "query": """
  FROM places-*
  | STATS count = count(name)
  | LIMIT 1
  """
}
```

A ES|QL API query can also include a filter using Elasticsearch DSL language:


```text
POST /_query?format=txt
{
  "query": """
  FROM places-*
  | STATS count = count(name)
  | LIMIT 1
  """,
  "filter": {
    "range": {
      "confidence": {
        "gte": 0.1,
        "lte": 1
      }
    }
  }
}
```

Other API endpoints available:

* [`_query/async`](https://www.elastic.co/docs/api/doc/elasticsearch/operation/operation-esql-async-query): start, stop, and get results asynchronously
* [`_query/queries`](https://www.elastic.co/docs/api/doc/elasticsearch/operation/operation-esql-list-queries): get details about running queries

## ES|QL sources: `ROW`, `SHOW`, `FROM`

In [8]:
# Creating a row directly, useful to test functions
esql('ROW a = 1, b = "two", c = null')

,a,b,c
0,1,two,NaN


In [9]:
# SHOW source returns Elasticsearch version
esql("SHOW INFO")

,version,date,hash
0,9.1.3,2025-08-24T22:05:04.526302670Z,0c781091a2f57de895a73a1391ff8426c0153c8d


In [10]:
# Basic query against all places indices
esql("FROM places-*")

,addresses.country,addresses.freeform,addresses.locality,addresses.postcode,addresses.region,alt_categories,brand,category,confidence,emails,geometry,id,name,phones,socials,source,updated,version,websites
0,ES,CALLE BISBE 8,Valencia,46002.0,NaN,NaN,NaN,retail,0.770000,NaN,POINT (-0.37205 39.47103),0cac5521-1719-43aa-afd2-2839d06035a9,STONECRIL .,960059724,NaN,Microsoft,2015-01-26T10:54:09.467Z,1,NaN
1,ES,"Carrer de Pintor Sorolla, 26",València,46002.0,NaN,fashion_accessories_store,NaN,womens_clothing_store,0.754817,NaN,POINT (-0.37142 39.47076),58e34780-dd72-43c0-ac6c-5f363be7b7c4,Maje,+34963520220,https://www.facebook.com/115176003187270,meta,2025-06-02T07:00:00.000Z,1,https://es.maje.com/
2,ES,"Calle Pintor Sorolla, 26",Valencia,46002.0,NaN,"[mens_clothing_store, shoe_store]",NaN,sports_wear,0.566292,NaN,POINT (-0.37123 39.47079),06d3c6fc-3fb3-4043-ac84-75df30a165e6,adidas Store,+34963512330,https://www.facebook.com/657973707600287,meta,2025-06-02T07:00:00.000Z,1,https://www.adidas.es/?utm_source=yext&utm_medium=referral&utm_campaign=ES210122_local
3,ES,"Carrer de Pintor Sorolla, 19",València,46002.0,NaN,cosmetic_and_beauty_supplies,NaN,beauty_and_spa,0.978451,NaN,POINT (-0.37117 39.47072),c0a81f99-413d-46ba-9861-d216978de0a3,MAC Cosmetics,+34963528071,https://www.facebook.com/129987917765329,meta,2025-06-02T07:00:00.000Z,1,https://www.maccosmetics.es/stores
4,ES,"Calle Pintor Sorolla, 26",Valencia,46002.0,NaN,clothing_store,NaN,mens_clothing_store,0.945926,NaN,POINT (-0.37117 39.47073),9cdd2cc3-47c8-4013-94f5-a184283237f4,Zadig&Voltaire,+34963525674,https://www.facebook.com/105047061709193,"[Microsoft, meta]",2025-06-02T07:00:00.000Z,1,https://storelocator.zadig-et-voltaire.com/zadigandvoltaire-ab3905a74ee5?utm_source=Yext&utm_medium=Facebook
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,HR,Ul. Vicka Andrića 3,NaN,21000.0,NaN,accommodation,NaN,hotel,0.941538,NaN,POINT (16.44158 43.50914),86375cb4-70db-4252-a317-6093f26efb3a,Pellegrini Luxury Rooms,+38521361800,https://www.facebook.com/939849692885071,meta,2025-06-02T07:00:00.000Z,1,NaN
996,HR,Poljička cesta,Split,21000.0,NaN,arts_and_entertainment,NaN,topic_concert_venue,0.242152,NaN,POINT (16.44156 43.50929),6c4c2893-f588-4078-a21f-338cb488fcbc,EVO RUKE,+385912517003,https://www.facebook.com/697039880437732,meta,2025-06-02T07:00:00.000Z,1,NaN
997,HR,NaN,NaN,NaN,NaN,"[attractions_and_activities, fountain]",NaN,landmark_and_historical_building,0.296943,NaN,POINT (16.44173 43.50872),688ac0a4-e3e0-48ee-a3fd-881671942d18,Mostar,NaN,https://www.facebook.com/318848615376235,meta,2025-06-02T07:00:00.000Z,1,NaN
998,HR,Hrvojeva 12,Split,21000.0,NaN,"[convenience_store, jewelry_store]",NaN,shopping,0.396144,NaN,POINT (16.44187 43.50876),d68a06e9-38d1-49bf-96c9-853c2de10d0e,Christmas Unique GIFTS,+385953904554,https://www.facebook.com/364461237457065,meta,2025-06-02T07:00:00.000Z,1,NaN


## Control the output: `LIMIT`

By default a ES|QL query result is limited to `1000` rows. Use `LIMIT` to reduce that number

In [11]:
# Basic query against all places indices,
# returing the first 5 rows (in no particular order)

esql("""
FROM places-*
| LIMIT 5
""")

,addresses.country,addresses.freeform,addresses.locality,addresses.postcode,addresses.region,alt_categories,brand,category,confidence,emails,geometry,id,name,phones,socials,source,updated,version,websites
0,ES,CALLE BISBE 8,Valencia,46002,NaN,NaN,NaN,retail,0.770000,NaN,POINT (-0.37205 39.47103),0cac5521-1719-43aa-afd2-2839d06035a9,STONECRIL .,960059724,NaN,Microsoft,2015-01-26T10:54:09.467Z,1,NaN
1,ES,"Carrer de Pintor Sorolla, 26",València,46002,NaN,fashion_accessories_store,NaN,womens_clothing_store,0.754817,NaN,POINT (-0.37142 39.47076),58e34780-dd72-43c0-ac6c-5f363be7b7c4,Maje,34963520220,https://www.facebook.com/115176003187270,meta,2025-06-02T07:00:00.000Z,1,https://es.maje.com/
2,ES,"Calle Pintor Sorolla, 26",Valencia,46002,NaN,"[mens_clothing_store, shoe_store]",NaN,sports_wear,0.566292,NaN,POINT (-0.37123 39.47079),06d3c6fc-3fb3-4043-ac84-75df30a165e6,adidas Store,34963512330,https://www.facebook.com/657973707600287,meta,2025-06-02T07:00:00.000Z,1,https://www.adidas.es/?utm_source=yext&utm_medium=referral&utm_campaign=ES210122_local
3,ES,"Carrer de Pintor Sorolla, 19",València,46002,NaN,cosmetic_and_beauty_supplies,NaN,beauty_and_spa,0.978451,NaN,POINT (-0.37117 39.47072),c0a81f99-413d-46ba-9861-d216978de0a3,MAC Cosmetics,34963528071,https://www.facebook.com/129987917765329,meta,2025-06-02T07:00:00.000Z,1,https://www.maccosmetics.es/stores
4,ES,"Calle Pintor Sorolla, 26",Valencia,46002,NaN,clothing_store,NaN,mens_clothing_store,0.945926,NaN,POINT (-0.37117 39.47073),9cdd2cc3-47c8-4013-94f5-a184283237f4,Zadig&Voltaire,34963525674,https://www.facebook.com/105047061709193,"[Microsoft, meta]",2025-06-02T07:00:00.000Z,1,https://storelocator.zadig-et-voltaire.com/zadigandvoltaire-ab3905a74ee5?utm_source=Yext&utm_medium=Facebook


## Change the output with `KEEP`, `RENAME`, and `SORT`

In [12]:
# Rename a field and only return a limited set of fields
esql("""
FROM places-*
| RENAME name as title
| KEEP title, category
| LIMIT 5
""")

,title,category
0,STONECRIL .,retail
1,Maje,womens_clothing_store
2,adidas Store,sports_wear
3,MAC Cosmetics,beauty_and_spa
4,Zadig&Voltaire,mens_clothing_store


In [13]:
# KEEP also establishes the order of the columns returned,
# sometimes relevant for post-processing in client code
esql("""
FROM places-*
| RENAME name as title
| KEEP category, title
| LIMIT 5
""")

,category,title
0,retail,STONECRIL .
1,womens_clothing_store,Maje
2,sports_wear,adidas Store
3,beauty_and_spa,MAC Cosmetics
4,mens_clothing_store,Zadig&Voltaire


In [14]:
# Once renamed, the previous identifier is not available anymore
esql("""
FROM places-*
| RENAME name as title
| KEEP name, category
| LIMIT 5
""")

Something went wrong!
verification_exception
Found 1 problem
line 3:8: Unknown column [name]


In [16]:
# Sort by a field, and reverse another
esql("""
FROM places-girona
| RENAME name AS title
| SORT category ASC, title DESC
| KEEP category, title
| LIMIT 5
""")

,category,title
0,abuse_and_addiction_treatment,Uphill Centre d'Assessorament Terapèutic
1,abuse_and_addiction_treatment,Forum Salud Mental Girona
2,abuse_and_addiction_treatment,Día 1 Centre Terapèutic
3,accommodation,‘El racó del bandoler’
4,accommodation,Îles Medes


## Include metadata with `METADATA`

Use `METADATA` to get access to the `_index` and `_id`:

In [17]:
# Get also the source index using the METADATA keyword
esql("""
FROM places-* METADATA _index, _id
| KEEP _index, _id, name, category
| LIMIT 5
""")

,_index,_id,name,category
0,places-girona,afd644d9-9b38-4cf4-9591-1f2bc5a452f7,Pantano De Canoves,active_life
1,places-girona,28d87f90-14e3-49f2-917a-a3f2c010324e,Embalse de Vallfornés,structure_and_geography
2,places-girona,f910bd47-4ea1-499d-bf60-0c402054a22f,Hotel Can Cuch,hotel
3,places-girona,eca6cb5e-4cc1-4af3-ab1c-ebfb5feb1480,El Bastión del Espino,landmark_and_historical_building
4,places-girona,7f061307-a767-46e4-a959-b87442196be1,Ermita de Sant Salvador,religious_organization


## Filtering

In [18]:
# A basic filter
esql("""
FROM places-* METADATA _index
| RENAME _index as dataset
| WHERE name LIKE "*Burger*" //Exact text search
    AND category IN ("restaurant", "burger_restaurant") //Exact keyword search
    AND confidence < 0.3
| SORT confidence DESC
| KEEP dataset, name, category, confidence
| LIMIT 5
""")

,dataset,name,category,confidence
0,places-bosnia,Burgers by Manzoni,burger_restaurant,0.296943
1,places-valencia,TORO Burger Lounge,restaurant,0.296943
2,places-bosnia,Burger Factory-Igalo,burger_restaurant,0.245690
3,places-bosnia,Smash Burgers Two Guys,burger_restaurant,0.242152
4,places-bosnia,Fajnl Tač Burgers,burger_restaurant,0.242152


Truly leveraging Elasticsearch full text search features with the `MATCH` function.

[ES|QL documentation](https://www.elastic.co/docs/reference/query-languages/esql/functions-operators/search-functions#esql-match) | [DSL documentation](https://www.elastic.co/docs/reference/query-languages/query-dsl/query-dsl-query-string-query#query-string-syntax)

In [19]:
# Full text search with MATCH function
esql("""
FROM places-*
| WHERE MATCH(name, "snack BAR charlie", {"operator": "AND"})
| KEEP name
| LIMIT 1000
""")

,name
0,Charlie Bar and Snack


Sending queries in [Lucene](https://lucene.apache.org/) syntax with the `QSTR` function.

[ES|QL documentation](https://www.elastic.co/docs/reference/query-languages/esql/functions-operators/search-functions#esql-qstr) | [DSL documentation](https://www.elastic.co/docs/reference/query-languages/query-dsl/query-dsl-query-string-query) | [Query String syntax](https://www.elastic.co/docs/reference/query-languages/query-dsl/query-dsl-query-string-query#query-string-syntax)

In [20]:
# Full text search with Lucene syntax
# https://www.elastic.co/docs/reference/query-languages/query-dsl/query-dsl-query-string-query#query-string-syntax
esql("""
FROM places-* METADATA _index
| WHERE QSTR("name: CHARLIE snack bar",
            {"fuzziness": 2,
            "minimum_should_match": 2,
            "phrase_slop": 3})
| KEEP _index, name
| LIMIT 1000
""")

,_index,name
0,places-bosnia,Charlie Bar
1,places-bosnia,Caffe Bar Charlie Chaplin
2,places-bosnia,Charlie Bar and Snack
3,places-bosnia,Caffee bar Charlie
4,places-bosnia,Charlie Tango Lounge Bar


---
>For more examples on how to do different text search operations with ES|QL check this [tutorial](https://www.elastic.co/guide/en/elasticsearch/reference/current/esql-search-tutorial.html)
---

## Aggregations with `STATS`

In [21]:
# STATS allows running aggrecations.
# In this count agg, no other data is available afterwards
esql("""
FROM ne_countries
| STATS counts = count(id)
""")

,counts
0,257


In [22]:
# When grouping by other fields, those are also available
# for further operations like sorting or filtering
esql("""
FROM ne_countries
| WHERE type in ("Country", "Sovereign country")
| STATS counts = count(id) BY continent
| WHERE counts > 30
| SORT continent
| KEEP continent, counts //not necessary, only used to define order
| LIMIT 5
""")

,continent,counts
0,Africa,53
1,Asia,48
2,Europe,48


In [23]:
# Aggregate: count by more than one grouping field
esql("""
FROM ne_countries
| WHERE type not in ("Country", "Sovereign country")
| STATS counts = count(id) BY continent, type
| WHERE counts > 1
| SORT continent, type
| KEEP continent, type, counts
| LIMIT 50
""")

,continent,type,counts
0,Africa,Indeterminate,2
1,Asia,Dependency,3
2,Asia,Indeterminate,5
3,Europe,Disputed,2
4,North America,Dependency,12
5,North America,Indeterminate,2
6,Oceania,Dependency,12
7,Seven seas (open ocean),Dependency,5
8,South America,Indeterminate,2


## Computing new fields with `EVAL`

In [24]:
# Use EVAL to compute new fields
esql("""
FROM ne_countries
| WHERE gdp_md IS NOT NULL
    AND pop_est > 0
   // AND type IN ("Country", "Sovereign country")
| EVAL gdp_pop = ROUND(gdp_md * 1e6 / ( pop_est::double))::integer
| SORT gdp_pop DESC
| KEEP name, type, gdp_md, pop_est, gdp_pop
| LIMIT 10
""")

,name,type,gdp_md,pop_est,gdp_pop
0,Monaco,Sovereign country,7188,38964,184478
1,Liechtenstein,Sovereign country,6876,38019,180857
2,Bermuda,Dependency,7484,63918,117088
3,Luxembourg,Sovereign country,71104,619896,114703
4,Fr. S. Antarctic Lands,Dependency,16,140,114286
5,Isle of Man,Country,7491,84584,88563
6,Cayman Is.,Dependency,5517,64948,84945
7,Macao,Country,53859,640445,84096
8,Falkland Is.,Disputed,282,3398,82990
9,Switzerland,Sovereign country,703082,8574832,81994


## Joins

Traditionally, Elasticsearch has not been able to join datasets in a comfortable way for developers and analysts. This has changed with ES|QL and with the introduction of the `index.mode: lookup` setting.

More details on:

* [`LOOKUP JOIN` docs](https://www.elastic.co/docs/reference/query-languages/esql/esql-lookup-join)
* [`index.mode`](https://www.elastic.co/docs/reference/elasticsearch/index-settings/index-modules#index-mode-setting)

Our `ne_countries` index was created with that setting so we can now join that dataset with our places indices.

In [25]:
# Let's find outlier data in our places-* indices
esql(
"""
FROM places-*

// Aggregate all our places by country
| STATS counts = count(addresses.country) BY addresses.country

// Filter for only those groups with counts between 1 and 100
| WHERE counts > 1 AND counts < 50

// The lookup common field needs to be the same
| RENAME addresses.country AS iso_a2, counts AS places

// Run the LOOKUP JOIN by the iso_a2 field
| LOOKUP JOIN ne_countries ON iso_a2

// Merge records for repeated iso_a2 entries
| STATS places = SUM(places), names = VALUES(name) BY iso_a2

// Sort and print
| SORT places DESC
| KEEP iso_a2, places, names
"""
)

,iso_a2,places,names
0,FR,30,"[France, Clipperton I.]"
1,HU,29,Hungary
2,SA,24,Saudi Arabia
3,AU,16,"[Australia, Indian Ocean Ter., Coral Sea Is., Ashmore and Cartier Is.]"
4,PL,15,Poland
5,SK,14,Slovakia
6,SI,12,Slovenia
7,US,11,United States of America
8,TR,11,Turkey
9,CZ,11,Czechia


## Wrap up

This is just the basics, there are plenty of functions and operators to process your datasets, please refer to the [documentation](https://www.elastic.co/docs/explore-analyze/query-filter/languages/esql) for further details.